In [25]:
import pandas as pd
import yfinance as yf

In [26]:
msft = yf.Ticker("MSFT")


In [27]:
def calculate_historical_growth_rate(financials):
    revenue = financials.income_stmt.loc['Total Revenue']
    growth_rates = revenue.pct_change().dropna()
    avg_growth_rate = growth_rates.mean()
    best_case = avg_growth_rate - growth_rates.std()
    worst_case = avg_growth_rate + growth_rates.std()
    return 1+(-1*avg_growth_rate), 1+(-1*best_case), 1+(-1*worst_case)

avg_growth_rate, best_case, worst_case = calculate_historical_growth_rate(msft)
print(f"Base Case: {avg_growth_rate}, Best Case: {best_case}, Worst Case: {worst_case}")


Base Case: 1.1173623672373567, Best Case: 1.163997306540595, Worst Case: 1.0707274279341181


# Revenue Projections

In [28]:
future_revenue = []
for i in range(1,6):
    future_revenue.append(merged_df.loc['Total Revenue'][merged_df.columns[0]]*avg_growth_rate**i)
future_revenue

[np.float64(273890098181.95535),
 np.float64(306034488467.4617),
 np.float64(341951420490.27655),
 np.float64(382083648679.19214),
 np.float64(426925890170.86865)]

# OPEX Projections as a percent of revenue

In [29]:
future_opex = []
opex_as_perc_of_rev = (merged_df.loc['Operating Expense'] / merged_df.loc['Total Revenue']).mean()
for i in range(5):
    future_opex.append((future_revenue[i] * opex_as_perc_of_rev))

future_opex

[np.float64(71771755213.06229),
 np.float64(80195058305.64737),
 np.float64(89606940189.13599),
 np.float64(100123422810.62923),
 np.float64(111874144727.59142)]

# Working Capital Changes: Projected as a percentage of revenue.


In [30]:
wc_as_perc_of_rev = (merged_df.loc['Working Capital'] / merged_df.loc['Total Revenue']).mean()
future_working_cap = [merged_df.loc['Working Capital'][-1]]
for i in range(5):
    future_working_cap.append(future_revenue[i]*wc_as_perc_of_rev)

future_change_working_cap = []


for i in range(len(future_working_cap) - 1):
    future_change_working_cap.append(future_working_cap[i+1] - future_working_cap[i])

future_change_working_cap

[np.float64(7091955512.001373),
 np.float64(9587796007.87178),
 np.float64(10713042443.944504),
 np.float64(11970350465.480103),
 np.float64(13375219132.769623)]

# Capital Expenditures: as a percent of revenue


In [31]:
future_capex = []
capex_as_perc_of_rev = (merged_df.loc['Capital Expenditure'] / merged_df.loc['Total Revenue']).mean()
for i in range(5):
    future_capex.append((future_revenue[i] * capex_as_perc_of_rev))

future_capex

[np.float64(-39673334247.73787),
 np.float64(-44329490671.25128),
 np.float64(-49532104634.85565),
 np.float64(-55345309689.05076),
 np.float64(-61840766249.642365)]

# Depreciation and Amortization: Typically based on historical ratios or specific asset schedules.



In [32]:
future_dep = []
avg_depreciation = (merged_df.loc['Reconciled Depreciation']).mean()
for i in range(5):
    future_dep.append(avg_depreciation)

future_dep

[np.float64(16869333333.333334),
 np.float64(16869333333.333334),
 np.float64(16869333333.333334),
 np.float64(16869333333.333334),
 np.float64(16869333333.333334)]

In [33]:
future_fcf = []
for i in range(5):
    ebit = future_revenue[i] - future_opex[i]
    FCF = (ebit * (1- merged_df.loc['Tax Rate For Calcs'][0])) + future_dep[i] - future_change_working_cap[i] + future_capex[i]
    future_fcf.append(FCF)
    
for i in range(len(future_fcf)):
    print(f"Year {i}: {int(future_fcf[i])}")


Year 0: 135436848122
Year 1: 147688700526
Year 2: 163041971140
Year 3: 180197137939
Year 4: 199365675723


In [34]:

def calculate_wacc(company):
    # Get the beta value
    beta = company.info.get('beta')
    if beta is None:
        raise ValueError("Beta value is not available for this company.")

    # Cost of Equity using CAPM
    risk_free_rate = 0.03
    market_return = 0.08
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

    # Cost of Debt
    interest_expense = company.financials.loc['Interest Expense'][0]
    total_debt = company.balance_sheet.loc['Total Debt'][0]
    if total_debt == 0:
        raise ValueError("Total debt is zero; cannot calculate cost of debt.")
    cost_of_debt = interest_expense / total_debt

    # Market Capitalization and Total Debt for Capital Structure
    market_cap = company.info.get('marketCap')
    if market_cap is None:
        raise ValueError("Market capitalization is not available for this company.")

    total_equity = market_cap
    total_debt = company.balance_sheet.loc['Total Debt'][0]
    total_value = total_equity + total_debt

    # Proportions of Equity and Debt
    equity_ratio = total_equity / total_value
    debt_ratio = total_debt / total_value

    # Tax Rate
    tax_rate = company.financials.loc["Tax Rate For Calcs"][0]

    # WACC Calculation
    wacc = (cost_of_equity * equity_ratio) + (cost_of_debt * debt_ratio * (1 - tax_rate))
    
    return wacc


calculate_wacc(msft)

0.07394173354009415

In [35]:
import requests
from bs4 import BeautifulSoup

# URL of the page to scrape
url = "https://stockanalysis.com/list/sp-500-stocks/"

# Send a GET request to the webpage
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Parse the webpage content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table containing the symbols
table = soup.find('table')  # Locate the first table on the page

if table:
    # Extract the symbols from the 2nd 'td' column (index 1)
    sp500 = [row.find_all('td')[1].text.strip() for row in table.find_all('tr')[1:]]

else:
    print("Table not found on the page.")



In [36]:
from DCF_Calculator import main

estimations = []
for ticker in sp500:
    try:
        actual_high = yf.Ticker(ticker).info.get('dayHigh')
        actual_low = yf.Ticker(ticker).info.get('dayLow')
        estimated_price = main(ticker, 5, '1')
        estimations.append((ticker,estimated_price,actual_high, actual_low))
    except Exception as e:
        print(f"{ticker}- {type(e).__name__}: {e}")

GOOG- KeyError: 'Net Debt'
GOOGL- KeyError: 'Net Debt'
META- KeyError: 'Net Debt'
BRK.B- KeyError: 'Total Revenue'
AVGO- ValueError: Beta value is not available for this company.
JPM- KeyError: "['Operating Expense'] not in index"
COST- KeyError: 'Net Debt'
BAC- KeyError: "['Operating Expense'] not in index"
CRM- KeyError: "['Interest Expense'] not in index"
AMD- KeyError: 'Net Debt'
ACN- KeyError: 'Net Debt'
WFC- KeyError: "['Operating Expense'] not in index"
AXP- KeyError: "['Operating Expense'] not in index"
ISRG- KeyError: "['Interest Expense'] not in index"
GS- KeyError: "['Operating Expense'] not in index"
MS- KeyError: "['Operating Expense'] not in index"
PGR- KeyError: "['Operating Expense'] not in index"
SYK- KeyError: "['Interest Expense'] not in index"
TJX- KeyError: 'Net Debt'
ELV- KeyError: "['Operating Expense'] not in index"
REGN- KeyError: 'Net Debt'
NKE- KeyError: "['Interest Expense'] not in index"
VRTX- KeyError: 'Net Debt'
PLD- KeyError: "['Capital Expenditure'] not

In [37]:
len(estimations)

385

In [38]:
estimations = pd.Series(estimations)
estimations[estimations.isna() == False]

0       (AAPL, 420.4263408775997, 225.24, 219.77)
1        (MSFT, 524.5091276089129, 410.65, 400.8)
2       (NVDA, 39.30452484499742, 108.15, 100.95)
3                    (AMZN, nan, 178.365, 171.25)
4       (LLY, 758.2797850921114, 927.9022, 893.0)
                          ...                    
380           (AAL, 6462.61201790886, 11.1, 10.7)
381        (IVZ, 330.6489991991721, 16.445, 15.7)
382       (PARA, 738.6490925092403, 10.42, 10.08)
383       (BBWI, 376.6879243918952, 29.58, 28.48)
384    (ETSY, 162.12476913763857, 56.1424, 53.12)
Length: 385, dtype: object

In [39]:
len(sp500)

503